<h1 align='center'><b><font color ='indianred'>Solution HW4: MultiClass Classification with PyTorch Lightning</b></h1>


- Fill the missing code indicated by # CODE HERE

- Submit the following two files:
  1.	FirstName_LastName_HW4.ipynb
  2.	FirstName_LastName_HW4.pdf (pdf version of the above file)

# <Font color = 'indianred'>**Specify Project Folder**

In [1]:
# add lines for autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys

if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

    base_folder = Path('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data') # Google Drive
    data_folder = Path('/content') # Keep data on Colab

    !pip install pytorch-lightning -U
    !pip install torchinfo -U

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 13.8 MB/s eta 0:00:00


In [3]:
base_folder

PosixPath('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data')

In [8]:
# Change the custom_function_folder to folder in your Google drive folder
# Make sure you keep the mlp_skip_two_layer.py and shared_utils.py files

custom_function_folder = Path('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Custom_files') # Your Google Drive

sys.path.append(str(custom_function_folder))
model_folder = Path('/content/drive/MyDrive/Colab_Notebooks/BUAN_6382_Applied_DeepLearning/Data') # Google drive folder where you want to save model and logs
model_folder.mkdir(parents=True, exist_ok=True)

# <Font color = 'indianred'>**Import Libraries**

In [9]:
# this should not be Google Drive Folder but local folder on Colab
data_folder

Path('/content')

In [13]:
import random
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchinfo import summary
from torchvision import transforms, datasets

import pytorch_lightning as pl
from pytorch_lightning.loggers import CSVLogger
import torchmetrics
from fastdownload import FastDownload

# The classea and functions we will import from .py files
# from mlp_skip_two_layer import PytorchMLPSkip
from shared_utils import check_loader, check_transforms, check_label_dist, show_confusion_matrix, compute_accuracy, plot_losses_acc


# <Font color = 'indianred'>**Imagenette Dataset**

For this HW, you will use the Imagenette dataset created by fastai https://github.com/fastai/imagenette).

Imagenette is a subset of 10 easily classified classes from Imagenet (tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute).

The dataset can be downloaded from this link: https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz

Dictionary to map the 10 classes with intergers
```
label_dict = {
    'n01440764' : 0,
    'n02102040' : 1,
    'n02979186' : 2,
    'n03000684' : 3,
    'n03028079' : 4,
    'n03394916' : 5,
    'n03417042' : 6,
    'n03425413' : 7,
    'n03445777' : 8,
    'n03888257' : 9
}

# Array to map integral indices with the actual class names
actual_label_dict = [
    'tench',
    'English springer',
    'cassette player',
    'chain saw',
    'church',
    'French horn',
    'garbage truck',
    'gas pump',
    'golf ball',
    'parachute'
]
```

# <Font color = 'indianred'>**Data Module**
Ceate a LightningDataModule for Imagenette2 Dataset
- Use the images in ```train``` folder to create ``training`` dataset
- Use the images in ``val`` folder to create both ```validation``` and ``test`` datasets.
- <Font color = 'indianred'>**Use 50% of the images in the ```val``` folder for validation dataset and 50% for test dataset.**</font>
- <Font color = 'indianred'>**Use a batch size of 64.**</font>
- The functions(methods) in the class below are mandatory. Add any other methods (functions) as to the class if required.
- Add the functionality to select a stratified random subset of validation and training sets for initial training (Hint see: modified data_loaders_dog_breed.py file on eLearning, please download the file again.)



In [ ]:
class ImagenetteDataModule(pl.LightningDataModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = model
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)

        # CODE HERE

    def prepare_data(self):

        # CODE HERE


    def setup(self, stage=None):
        # CODE HERE


    def train_dataloader(self):
        return # CODE HERE

    def val_dataloader(self):
        return # CODE HERE

    def test_dataloader(self):
        return # CODE HERE

# <Font color = 'indianred'>**Data Transforms**

In [ ]:
# DO NOT CHANGE THIS CELL
# YOU HAVE TO USE THESE TRANSFORMATIONS
# USE data_transforms['TRAIN] FOR TRAINING SET AND data_transforms['test] FOR BOTH VALIDATION AND TEST SET
data_transforms = {
    "train": transforms.Compose(
        [
            # Resize the shorter side of the image to (500, 375) pixels
            transforms.Resize((500, 375)),
            # Convert image to PyTorch tensor
            transforms.ToTensor(),
            # Normalize tensor values to range [-1, 1]
            transforms.Normalize((0.5,), (0.5,)),
        ]
    ),
    "test": transforms.Compose(
        [
            # Resize the shorter side of the image to 640 pixels
            transforms.Resize((500, 375)),
            # Convert image to PyTorch tensor
            transforms.ToTensor(),
            # Normalize tensor values to range [-1, 1]
            transforms.Normalize((0.5,), (0.5,)),
        ]
    ),
}

# <Font color = 'indianred'>**Instantiate Data Module**
- <Font color = 'indianred'>**Use 50% of the images in the ```val``` folder for validation dataset and 50% for test dataset.**</font>
-  <Font color = 'indianred'>**Use a samll subset (1000 samples) for both validation and training dataset** </font>
- <Font color = 'indianred'>**Use a batch size of 64.**</font>


In [ ]:
torch.manual_seed(123)
# Use batch size of 64
# Select only 1000
dm = # CODE HERE

In [ ]:
dm.prepare_data()

In [ ]:
dm.setup(stage='fit')
check_loader(dm.train_dataloader())

In [ ]:
len(dm.train_dataloader())

# <font color = 'indianred'>**LightningModule**

- <font color = 'indianred'>**Use SGD optimizer.** </font>
- <font color = 'indianred'>**Print accuracy and loss at the end of every epoch.**</font>
- <font color = 'indianred'>  **Log loss and accuracy at the end of each epoch or both training and validation.**</font>
- <font color = 'indianred'> **Also log training loss at every 10 steps.**</font>


In [ ]:
class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = model
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)

    def forward(self, x):
        return # CODE HERE

    def _shared_step(self, batch):
      # CODE HERE


    def training_step(self, batch, batch_idx):
      # CODE HERE


    def validation_step(self, batch, batch_idx):
      # CODE HERE


    def on_train_epoch_end(self):
      # CODE HERE


    def on_validation_epoch_end(self):
      # CODE HERE


    def configure_optimizers(self):
      # CODE HERE


    def test_step(self, batch, batch_idx):
      # CODE HERE


In [ ]:
# DO NOT CHANGE THIS CELL
# Define the model architecture and training parameters
num_features = 3*500*375
hidden_dim1 = 300
hidden_dim2 = 200
hidden_dim3 = 100
num_classes = 10
epochs = 10
learning_rate = 0.03


# Set a random seed for reproducibility
torch.manual_seed(42)

# Create the neural network model
model = PytorchMLPSkip(num_features, hidden_dim1, hidden_dim2, hidden_dim3, num_classes)

lightning_model = LightningModel(model=model, learning_rate=learning_rate)


# Determine the computing device (CPU or GPU) to use
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

summary(model, (1, 3, 500, 375))

In [ ]:
# DO NOT CHANGE THIS CELL
trainer = pl.Trainer(
        max_epochs=10,
        accelerator="auto",  # set to "auto" or "gpu" to use GPUs if available
        devices="auto",  # Uses all available GPUs if applicable
        deterministic=True,
        log_every_n_steps = 10,
        logger=CSVLogger(save_dir=model_folder/'logs', name="skip_two_layer")
    )

In [ ]:
# DO NOT CHANGE THIS CELL
trainer.fit(
        model=lightning_model,
        datamodule=dm,
    )

## <Font color = 'indianred'>**Plot Losses**

In [ ]:
# DO NOT CHANGE THIS CELL
file = f"{trainer.logger.log_dir}/metrics.csv"
file


In [ ]:
# DO NOT CHANGE THIS CELL
plot_losses_acc(file)